In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
antipsychotic_data = (pd.read_excel('data/ASD_Antipsychotics_MPH_MasterData_V3.xlsx', 
                                    sheetname='Antipsychotics_data', 
                                    na_values=['ND','NR','NRNR']))

In [3]:
antipsychotic_data.head()

,REFID,Citation,Child Paper,Rx Grouping,Group_Desc,Intervention_category,Dose,Route,Rx_Durn,OUTC_Main_ CATG,...,Change score_Q1,Change score_Q3,Change score Min,Change score Max,Change score 95% L,Change score 95% H,Other stats,Results,Comments,Comments_2
0,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.2",NaN,NaN
1,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.0",NaN,NaN
3,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=0.8",NaN,NaN


In [4]:
antipsychotic_data.columns

Index(['REFID', 'Citation', 'Child Paper', 'Rx Grouping', 'Group_Desc',
       'Intervention_category', 'Dose', 'Route', 'Rx_Durn', 'OUTC_Main_\nCATG',
       'Outcome_Sub_category', 'Outc_Tool', 'Outcome_specify',
       'REPORTER_RATER', 'Baseline sample size', 'BL_count', 'BL %', 'BL Mean',
       'BL SD', 'BL SE', 'BL_Median', 'BL_Q1', 'BL_Q3', 'BL Min', 'BL Max',
       'BL 95% L', 'BL 95% H', 'Assessment tmpt', 'sample_size', 'count', ' %',
       'Mean', 'SD', 'SE', 'Median', 'Q1', 'Q3', 'Min', 'Max', '95% L',
       '95% H', 'FUP timepoint ', 'FUP_sample_size', 'FUP_count ',
       'FUP _percent ', 'FUP\n Mean', 'FUP \nSD', 'FUP \n SE', 'FUP \n Median',
       'FUP\n _Q1', 'FUP \n_Q3', 'FUP\n  Min', 'FUP \n Max', 'FUP \n 95% L',
       'FUP \n 95% H', 'Change score _Describe', 'Change score sample_size',
       'Change score_count', 'Change score %', 'Change score Mean',
       'Change score SD', 'Change score SE', 'Change score_\nMedian',
       'Change score_Q1', 'Change scor

In [6]:
antipsychotic_data.groupby('Outc_Tool').Outc_Tool.count().sort_values(ascending=False)

Outc_Tool
Aberrant Behavior Checklist                           215
clinical global impression (overall, irritability)    101
Other                                                  28
Children’s Yale-Brown Obsessive Compulsive Scale       26
Vineland Adaptive Behavior Scale                       22
Ritvo-Freeman Real Life Rating Scales                  12
Modiﬁed Real Life Rating Scale for Autism (M-RLRS      10
Pediatric  Quality of  Life  Inventory  (PedsQL)        8
Children’s Global Assessment Scale (CGAS)               4
CARS                                                    4
Multiple                                                2
Name: Outc_Tool, dtype: int64

In [7]:
(antipsychotic_data[antipsychotic_data.Outc_Tool=='Aberrant Behavior Checklist']
             .groupby('REFID')
             .agg({'Baseline sample size' : np.sum, 
                   'sample_size' : np.sum,
                   'FUP_sample_size': np.sum,
                   'Change score sample_size': np.sum}))

,sample_size,FUP_sample_size,Change score sample_size,Baseline sample size
REFID,,,,
204_805,NaN,NaN,395.0,568.0
4398_4387,539.0,NaN,385.0,962.0
4412_4405_4400_4393_4396_4395_4391_4380_1483_412_1605_1745,820.0,406.0,NaN,1226.0
4423_412_1745,1610.0,NaN,NaN,1860.0
4424_4425_840_2524_1111_713_869,NaN,NaN,3800.0,4718.0
83,205.0,NaN,205.0,425.0
90,500.0,NaN,NaN,590.0


In [8]:
(antipsychotic_data[antipsychotic_data.Outc_Tool=='clinical global impression (overall, irritability)']
             .groupby('REFID')
             .agg({'Baseline sample size' : np.sum, 
                   'sample_size' : np.sum,
                   'FUP_sample_size': np.sum,
                   'Change score sample_size': np.sum}))

,sample_size,FUP_sample_size,Change score sample_size,Baseline sample size
REFID,,,,
4388,156.0,NaN,NaN,156.0
204_805,92.0,NaN,92.0,184.0
4398_4387,NaN,NaN,NaN,55.0
4412_4405_4400_4393_4396_4395_4391_4380_1483_412_1605_1745,1139.0,81.0,NaN,1708.0
4423_412_1745,NaN,NaN,NaN,496.0
4424_4425_840_2524_1111_713_869,375.0,NaN,452.0,1110.0
83,41.0,NaN,NaN,85.0
90,224.0,NaN,NaN,236.0


In [9]:
antipsychotic_data.head()

,REFID,Citation,Child Paper,Rx Grouping,Group_Desc,Intervention_category,Dose,Route,Rx_Durn,OUTC_Main_ CATG,...,Change score_Q1,Change score_Q3,Change score Min,Change score Max,Change score 95% L,Change score 95% H,Other stats,Results,Comments,Comments_2
0,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.2",NaN,NaN
1,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.0",NaN,NaN
3,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=0.8",NaN,NaN


Extract columns and rows of interest

In [10]:
analysis_subset = ['REFID', 'Group_Desc', 'Intervention_category', 'Dose', 'Route', 'Rx_Durn', 'OUTC_Main_\nCATG',
       'Outcome_Sub_category', 'Outc_Tool', 'Outcome_specify',
       'Baseline sample size', 'BL Mean', 'BL SD', 'sample_size', 
       'Mean', 'SD', 'Change score sample_size', 'Change score Mean', 'Change score SE']

In [12]:
cgi_subset = antipsychotic_data.loc[antipsychotic_data.Outc_Tool=='clinical global impression (overall, irritability)',
                                   analysis_subset]
abc_subset = antipsychotic_data.loc[antipsychotic_data.Outc_Tool=='Aberrant Behavior Checklist',
                                   analysis_subset]

## ABC meta-analysis

In [13]:
abc_subset.head()

,REFID,Group_Desc,Intervention_category,Dose,Route,Rx_Durn,OUTC_Main_ CATG,Outcome_Sub_category,Outc_Tool,Outcome_specify,Baseline sample size,BL Mean,BL SD,sample_size,Mean,SD,Change score sample_size,Change score Mean,Change score SE
0,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,ASD symptom severity,Aberrant Behavior Checklist,Aberrant Behavior Checklist - Irritability,49.0,26.2,7.9,49.0,11.3,7.4,NaN,NaN,NaN
1,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,ASD symptom severity,Aberrant Behavior Checklist,Aberrant Behavior Checklist - Irritability,52.0,25.5,6.6,52.0,21.9,9.5,NaN,NaN,NaN
2,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,ASD symptom severity,Aberrant Behavior Checklist,Aberrant Behavior Checklist - Hyperactivity/No...,49.0,31.8,9.6,49.0,17.0,9.7,NaN,NaN,NaN
3,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,ASD symptom severity,Aberrant Behavior Checklist,Aberrant Behavior Checklist - Hyperactivity/No...,52.0,32.3,8.5,52.0,27.6,10.6,NaN,NaN,NaN
4,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,ASD symptom severity,Aberrant Behavior Checklist,Aberrant Behavior Checklist - Stereotypic Beha...,49.0,10.6,4.9,49.0,5.8,4.6,NaN,NaN,NaN


In [14]:
abc_subset.Outcome_specify.value_counts()

Aberrant Behavior Checklist - Irritability                                               43
Aberrant Behavior Checklist - Stereotypic Behavior                                       29
Aberrant Behavior Checklist - Inappropriate Speech                                       25
Aberrant Behavior Checklist - Hyperactivity/Noncompliance                                20
Aberrant Behavior Checklist - Social Withdrawal                                          16
Aberrant Behavior Checklist - Social Withdrawal/Lethargy                                 12
Aberrant Behavior Checklist - Hyperactivity                                              11
Aberrent Behavior Checklist - Inappropriate Speech                                        7
Aberrent Behavior Checklist - irritability                                                7
Aberrent Behavior Checklist - hyperactivity                                               7
Aberrent Behavior Checklist - Social Withdrawal/Lethargy                        

In [15]:
abc_subset.Dose.value_counts()

2-15 mg/kg                                  25
1 mg/mL                                     22
up to 2.5 mg/day                            20
up to 3.5 mg/day + mean of 10.9 sessions    15
up to 3.5 mg/day                            15
0.02 mg/kg/day                              10
5 mg/kg                                     10
10 mg/kg                                    10
1.25-10mg/day                               10
0.25-3mg/day                                10
15 mg/kg                                    10
1.25 - 1.75 mg/day                           6
0.125 - 0.175 mg/day                         6
2 - 15mg/day                                 5
2.5 mg/day                                   5
Name: Dose, dtype: int64

In [18]:
pd.pivot_table?

In [27]:
abc_subset.pivot_table(values='Baseline sample size', columns='Intervention_category', index='Outcome_specify', 
                      aggfunc='sum').fillna(0).astype(int)

Intervention_category,Medical,Medical + parent training,Placebo,other,placebo
Outcome_specify,,,,,
Aberrant Behavior Checklist - Hyperactivity,442,0,167,84,0
Aberrant Behavior Checklist - Hyperactivity/Noncompliance,525,225,218,24,0
Aberrant Behavior Checklist - Inappropriate Speech,850,225,320,84,0
Aberrant Behavior Checklist - Inappropriate speech,121,0,66,27,0
Aberrant Behavior Checklist - Irritability,1113,225,556,111,0
Aberrant Behavior Checklist - Lethargy/Social Withdrawal,169,0,69,84,0
Aberrant Behavior Checklist - Social Withdrawal,462,225,150,27,0
Aberrant Behavior Checklist - Social Withdrawal/Lethargy,340,0,167,0,0
Aberrant Behavior Checklist - Stereotypic Behavior,907,225,386,109,0
